In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from sktime.datasets import load_tsf_to_dataframe
import os

In [2]:
# --- Paths ---
tsf_file_path = "data/kaggle_web_traffic_dataset_with_missing_values.tsf"
csv_file_path = "data/web_traffic_converted.csv"

In [ ]:
# --- Conversion using sktime ---
# The load_tsf_to_dataframe function handles the complex TSF parsing for you.
# It returns a hierarchical Pandas DataFrame, which we will flatten slightly.
try:
    # Load the TSF data using the sktime utility
    pandas_df, metadata = load_tsf_to_dataframe(
        tsf_file_path, 
        return_type='pd_multiindex_hier' # Recommended for most time-series data
    )

    # The resulting DataFrame has a multi-index (series_id, timestamp).
    # To get a flat, Spark-friendly table, we reset the index.
    pandas_df = pandas_df.reset_index()

    # Rename the value column, which sktime names 'series_value' by default
    pandas_df = pandas_df.rename(columns={'series_value': 'TrafficCount', 'level_1': 'Timestamp'})

    # --- Save to CSV ---
    pandas_df.to_csv(csv_file_path, index=False)

    print(f"File successfully converted and saved to {csv_file_path}")
    print("\nFirst 5 rows of the converted Pandas DataFrame:")
    print(pandas_df.head())

except FileNotFoundError:
    print(f"Error: The file path was not found: {tsf_file_path}")
except Exception as e:
    print(f"An error occurred during TSF loading: {e}")

In [5]:
df = pd.read_csv(csv_file_path)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116485589 entries, 0 to 116485588
Data columns (total 3 columns):
 #   Column        Dtype         
---  ------        -----         
 0   series_name   object        
 1   timestamp     datetime64[ns]
 2   TrafficCount  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 2.6+ GB


In [6]:
print(f"min date: {df['timestamp'].min()}\nmax date: {df['timestamp'].max()}" )

min date: 2015-07-01 00:00:00
max date: 2017-09-10 00:00:00


In [7]:
# get 3 month prior to max date
split_date = pd.Timestamp(df['timestamp'].max()) - relativedelta(months=3)
print(f"split date: {split_date}")

# time series train/test split
train_df = df[df['timestamp'] < split_date]
test_df = df[df['timestamp'] >= split_date]

print(f"train max date: {train_df['timestamp'].max()}\ntest min date: {test_df['timestamp'].min()}\n")

split date: 2017-06-10 00:00:00
train max date: 2017-06-09 00:00:00
test min date: 2017-06-10 00:00:00



In [8]:
print(f"Training records: {len(train_df)}")
print(f"Testinging records: {len(test_df)}")

Training records: 102994730
Testinging records: 13490859


In [11]:
train_df.to_csv('data/web_traffic_train.csv', index=False)
test_df.to_csv('data/web_traffic_test.csv', index=False)

In [12]:
t_df = pd.read_csv('data/web_traffic_train.csv')
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102994730 entries, 0 to 102994729
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   series_name   object 
 1   timestamp     object 
 2   TrafficCount  float64
dtypes: float64(1), object(2)
memory usage: 2.3+ GB
